In [14]:
import tensorflow as tf
import pandas as pd
import time 

print(tf.__version__)

import os
os.chdir('C:\\Users\\kritanu\\Documents')
os.getcwd()

1.6.0-rc1


'C:\\Users\\kritanu\\Documents'

In [6]:
mnist_train = pd.read_csv("C:\\Users\\kritanu\\Documents\\train.csv")
mnist_test = pd.read_csv("C:\\Users\\kritanu\\Documents\\test.csv")

raw_data = np.loadtxt(mnist_train, skiprows=1, dtype='int', delimiter=',')

X_train, X_test, y_train, y_test = train_test_split(raw_data[:,1:], raw_data[:,0], test_size=0.1)

In [7]:
#Optimization variables

learning_rate = 0.0001
epochs = 20
batch_size = 100
beta = 0.01 #regularizer

#training data placeholders
# input x = 28 x 28 pixels = 784 flattened image

#mnist.train.nextbatch()

x = tf.placeholder(tf.float32, [None, 784])

#dynamically reshape the input

x_shaped = tf.reshape(x, [-1, 28, 28, 1])

#output data placeholder

y = tf.placeholder(tf.float32, [None, 10])

#dropout prob placeholder

keep_prob = tf.placeholder(tf.float32)

In [8]:
#function to create layers for model

def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    
    #make the filter input shape for tf.nn.conv_2d
    
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]
    
    #initialization of w and b for the filter
    
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev = 0.03), name= name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name = name+'_b')
    
    #setup conv layer operation
    
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding = "SAME")
    
    #adding the bias
    
    out_layer += bias
    
    #relu activation
    
    out_layer = tf.nn.relu(out_layer)
    
    #max-pooling
    
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize = ksize, strides = strides, padding = "SAME")
    
    return out_layer

In [9]:
#creating some conv layers

layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')

In [10]:
#FC layer
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

In [11]:
#setup some weights and bias values for 1st layer, then activate with ReLU

wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')

dense_layer1 = tf.matmul(flattened, wd1) + bd1

dense_layer1 = tf.nn.relu(dense_layer1)

#drop out
dense_layer1 = tf.nn.dropout(dense_layer1, keep_prob)

In [12]:
#Another layer with softmax activation

wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')

dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2

y_ = tf.nn.softmax(dense_layer2)

regularizers = tf.nn.l2_loss(wd2) + tf.nn.l2_loss(bd2)

In [13]:
#Optimization function 

#cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense_layer2, labels=y))

#WITH REGULARIZATION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=dense_layer2, labels=y) + beta * regularizers)

In [13]:
start_time = time.time()

#Optimizer
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.control_dependencies(update_ops):
        # Ensures that we execute the update_ops before performing the train_step
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)
#Accuracy

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Setup the initialization operator

init_op = tf.global_variables_initializer()
        

with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    writer = tf.summary.FileWriter("output", sess.graph)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y, keep_prob : 0.7})
            avg_cost += c / total_batch
            
        test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob : 1.0})*100
        
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), " test accuracy: {:.3f}".format(test_acc))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob : 1.0})*100)
    print("--- %s seconds ---" % (time.time() - start_time))
    writer.close()

Epoch: 1 cost = 1.157  test accuracy: 90.830
Epoch: 2 cost = 0.301  test accuracy: 94.870
Epoch: 3 cost = 0.201  test accuracy: 96.280
Epoch: 4 cost = 0.152  test accuracy: 97.300
Epoch: 5 cost = 0.126  test accuracy: 97.580
Epoch: 6 cost = 0.108  test accuracy: 97.750
Epoch: 7 cost = 0.099  test accuracy: 98.130
Epoch: 8 cost = 0.089  test accuracy: 98.300
Epoch: 9 cost = 0.081  test accuracy: 98.360
Epoch: 10 cost = 0.075  test accuracy: 98.490
Epoch: 11 cost = 0.070  test accuracy: 98.640
Epoch: 12 cost = 0.064  test accuracy: 98.860
Epoch: 13 cost = 0.061  test accuracy: 98.850
Epoch: 14 cost = 0.057  test accuracy: 98.750
Epoch: 15 cost = 0.053  test accuracy: 98.850
Epoch: 16 cost = 0.051  test accuracy: 98.940
Epoch: 17 cost = 0.047  test accuracy: 98.690
Epoch: 18 cost = 0.044  test accuracy: 99.040
Epoch: 19 cost = 0.043  test accuracy: 98.940
Epoch: 20 cost = 0.041  test accuracy: 99.060
99.0599989891
--- 204.50387406349182 seconds ---


## Proc - i5-7700HQ
## gpu - 1050ti
## Py - 3.6.4
## Tensorflow - 1.6.0-rc1 gpu
## Cuda Toolkit - 9.0